In [ ]:
#!sudo apt-get install libpq-dev -y
!pip install psycopg2

# General psql queries

In [ ]:
import psycopg2

In [ ]:
#открываем соединение
conn_string="dbname='{}' user='{}' host='{}' password='{}'".format("titanik_db{}", "student{}",
                                                                  "",
                                                                  "student{}")
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Создаем таблицы в базе

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("../lecture_05_ml/notebooks/titanik.tsv", sep='|', index_col=0)
data['prename'] = data['name'].apply(lambda x: x.split()[1].strip("."))
data['pclass'] = data['pclass'].apply(str)
data['age'].fillna(0, inplace=True)
data['age'] = data['age'].apply(float)
data['fare'].fillna(0, inplace=True)
data['fare'] = data['fare'].apply(float)
data['embarked'].fillna("None", inplace=True)

lbl_embarked = LabelEncoder().fit(data['embarked'])
lbl_pclass = LabelEncoder().fit(data['pclass'])
lbl_prename = LabelEncoder().fit(data['prename'])
lbl_sex = LabelEncoder().fit(data['sex'])

data['embarked'] = lbl_embarked.transform(data['embarked'])
data['pclass'] = lbl_pclass.transform(data['pclass'])
data['prename'] = lbl_prename.transform(data['prename'])
data['sex'] = lbl_sex.transform(data['sex'])

data.head()

In [ ]:
# Создадим впомогательные таблицы и заполним их
create_additional = '''
        CREATE TABLE embarked (
        embarked_id serial PRIMARY KEY,
        embarked_type VARCHAR(10) NOT NULL UNIQUE
        );
        CREATE TABLE pclass (
        pclass_id serial PRIMARY KEY,
        pclass_type VARCHAR(10) NOT NULL UNIQUE
        );
        '''
cursor.execute(create_additional)
conn.commit()

In [ ]:
insert_query = '''INSERT INTO embarked (embarked_id, embarked_type) VALUES (%s, %s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_embarked.classes_)), lbl_embarked.classes_)))
conn.commit()
insert_query = '''INSERT INTO pclass (pclass_id, pclass_type) VALUES (%s, %s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_pclass.classes_)), lbl_pclass.classes_)))
conn.commit()

In [ ]:
#TODO Создайте таблицу для prename (и заполните её)

In [ ]:
#TODO Создайте таблицу для пола

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
#TODO создать основную таблицу

In [ ]:
#TODO вставьте данные из таблицы

In [ ]:
#TODO Вопрос Кого больше мужчин или женщин?

In [ ]:
#TODO Вопрос Какой процент мужчин выжил и какой процент женщин?

In [ ]:
#TODO Что вы можете сказать выживаемости пассижиров разного класса и пола?